In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# Imported weights were generated from fitting model in Google Collaboratory notebook.
# Mount google drive
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import numpy as np
import h5py

def import_hdf5(file_name, dataset_name):
  hdf5 = h5py.File('drive/Udacity/dog-project/saved_models/' + file_name + '.h5', 'r')
  import_data = hdf5.get(dataset_name)
  return np.array(import_data)

In [2]:
# Import train, validation, and test data.

train_tensors = import_hdf5('train_data', 'train_tensors')
valid_tensors = import_hdf5('valid_data', 'valid_tensors')
test_tensors = import_hdf5('test_data', 'test_tensors')

print(train_tensors.shape)
print(valid_tensors.shape)
print(test_tensors.shape)

(6680, 224, 224, 3)
(835, 224, 224, 3)
(836, 224, 224, 3)


In [3]:
train_targets = import_hdf5('train_targets', 'y_train')
valid_targets = import_hdf5('valid_targets', 'y_valid')
test_targets = import_hdf5('test_targets', 'y_test')

print(train_targets.shape)
print(valid_targets.shape)
print(test_targets.shape)
#print(valid_targets[:5])

(6680, 133)
(835, 133)
(836, 133)


In [0]:
train_targets = train_targets.astype(int)
valid_targets = valid_targets.astype(int)
test_targets = test_targets.astype(int)

In [5]:
# Build model architecture

!pip install -q keras

import keras
from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential, Model

model = Sequential()
model.add(Conv2D(input_shape=train_tensors.shape[1:], kernel_size=3, filters=32, strides=1, activation='relu'))
model.add(Conv2D(kernel_size=3, filters=32, strides=1, activation='relu'))
model.add(Conv2D(kernel_size=3, filters=32, strides=1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(kernel_size=3, filters=64, strides=1, activation='relu'))
model.add(Conv2D(kernel_size=3, filters=64, strides=1, activation='relu'))

# Classification block
model.add(GlobalAveragePooling2D())
model.add(Dense(133, activation='softmax', name='predictions'))   
  
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 32)      9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 218, 218, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 109, 109, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 107, 107, 64)      18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 105, 105, 64)      36928     
_________________________________________________________________
global_average_pooling2d_1 ( (None, 64)                0         
__________

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 10

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='drive/Udacity/dog-project/saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/10
6680/6680 [==============================] - 90s 13ms/step - loss: 4.8856 - acc: 0.0084 - val_loss: 4.8650 - val_acc: 0.0168

Epoch 00001: val_loss improved from inf to 4.86499, saving model to drive/Udacity/dog-project/saved_models/weights.best.from_scratch.hdf5
Epoch 2/10
2500/6680 [==========>...................] - ETA: 51s - loss: 4.8514 - acc: 0.0184

6680/6680 [==============================] - 86s 13ms/step - loss: 4.8416 - acc: 0.0174 - val_loss: 4.8128 - val_acc: 0.0192

Epoch 00002: val_loss improved from 4.86499 to 4.81282, saving model to drive/Udacity/dog-project/saved_models/weights.best.from_scratch.hdf5
Epoch 3/10
4520/6680 [===================>..........] - ETA: 26s - loss: 4.7944 - acc: 0.0170

6680/6680 [==============================] - 86s 13ms/step - loss: 4.7921 - acc: 0.0171 - val_loss: 4.8244 - val_acc: 0.0156

Epoch 00003: val_loss did not improve
Epoch 4/10
6680/6680 [==============================] - 85s 13ms/step - loss: 4.7549 - acc: 0.0190 - val_loss: 4.7429 - val_acc: 0.0168

Epoch 00004: val_loss improved from 4.81282 to 4.74290, saving model to drive/Udacity/dog-project/saved_models/weights.best.from_scratch.hdf5
Epoch 5/10
 140/6680 [..............................] - ETA: 1:19 - loss: 4.7268 - acc: 0.0143

6680/6680 [==============================] - 86s 13ms/step - loss: 4.7033 - acc: 0.0238 - val_loss: 4.7122 - val_acc: 0.0311

Epoch 00005: val_loss improved from 4.74290 to 4.71225, saving model to drive/Udacity/dog-project/saved_models/weights.best.from_scratch.hdf5
Epoch 6/10
4000/6680 [================>.............] - ETA: 32s - loss: 4.6545 - acc: 0.0290

6680/6680 [==============================] - 86s 13ms/step - loss: 4.6545 - acc: 0.0298 - val_loss: 4.7181 - val_acc: 0.0311

Epoch 00006: val_loss did not improve
Epoch 7/10
6680/6680 [==============================] - 86s 13ms/step - loss: 4.6027 - acc: 0.0353 - val_loss: 4.6120 - val_acc: 0.0359

Epoch 00007: val_loss improved from 4.71225 to 4.61197, saving model to drive/Udacity/dog-project/saved_models/weights.best.from_scratch.hdf5
Epoch 8/10
  60/6680 [..............................] - ETA: 1:21 - loss: 4.5722 - acc: 0.0333

6680/6680 [==============================] - 86s 13ms/step - loss: 4.5489 - acc: 0.0440 - val_loss: 4.5646 - val_acc: 0.0407

Epoch 00008: val_loss improved from 4.61197 to 4.56458, saving model to drive/Udacity/dog-project/saved_models/weights.best.from_scratch.hdf5
Epoch 9/10
3980/6680 [================>.............] - ETA: 33s - loss: 4.5201 - acc: 0.0508

6680/6680 [==============================] - 86s 13ms/step - loss: 4.5095 - acc: 0.0507 - val_loss: 4.5410 - val_acc: 0.0431

Epoch 00009: val_loss improved from 4.56458 to 4.54097, saving model to drive/Udacity/dog-project/saved_models/weights.best.from_scratch.hdf5
Epoch 10/10
4860/6680 [====================>.........] - ETA: 22s - loss: 4.4544 - acc: 0.0521

6680/6680 [==============================] - 86s 13ms/step - loss: 4.4528 - acc: 0.0497 - val_loss: 4.5559 - val_acc: 0.0419

Epoch 00010: val_loss did not improve


In [0]:
model.load_weights('drive/Udacity/dog-project/saved_models/weights.best.from_scratch.hdf5')

In [9]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 4.4258%
